In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import train_test_split, LeaveOneOut, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.feature_selection import SelectFromModel
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
import pylab
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Data frame
df = pd.read_csv("C:/Users/madan/projet/all indic.csv",  encoding = "utf-8", delimiter=';', header=0, decimal='.')
print('Dataframe shape: ', df.shape)

#df.columns.tolist()
df.head(2)


df_fin =  df.drop(['CRM',
 'Adresse',
 'latitude','longitude'], axis=1) 

df_fin = df_fin.set_index('Creche nom complet')

print('Dataframe after removing unwanted columns: ', df_fin.shape)


### Loading and summarizing the dataset
### Count, mean, standard deviation, minimum, 25 50 75 percentiles, and maximum
    
df_fin.describe()

# plot the histograms to have a quick look at the distributions

#  plot Q-Q plots to visualise if the variable is normally distributed

def diagnostic_plots(df, variable):
    # function to plot a histogram and a Q-Q plot
    # side by side, for a certain variable
    
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    df[variable].hist()
    
    plt.subplot(1, 2, 2)
    stats.probplot(df[variable], dist="norm", plot=pylab)
    print(variable)
    plt.show()
    

     
test_variables = [ 'X', 'Z']

# Check variable distribution
for var in trans_variables:
    diagnostic_plots(data, var)
    
    
print('Dataframe shape: ', df.shape)
# Quantile method for skewed distribution
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

df_out = remove_outlier(df, 'CA')

print('Dataframe after removing outlier: ', df_out.shape)

print('Dataframe after removing outlier: ', df_out.head(2))



# Get categorical columns less than 15 cat
cat_cols = []
for col in df_out.columns:
    if df_out[col].dtypes =='O': # if variable  is categorical
        if len(df_out[col].unique())<15: # and has more than 10 categories
            cat_cols.append(col)  # add to the list
            print(df_out.groupby(col)[col].count()/np.float(len(df_out))) # and print the percentage of observations within each category
            print()

# Get categorical columns less than 150 cat
cat_cols_all = []
for col in df_out.columns:
    if df_out[col].dtypes =='O': # if variable  is categorical
        if len(df_out[col].unique())<150: # and has more than 10 categories
            cat_cols_all.append(col)  # add to the list
            print(df_out.groupby(col)[col].count()/np.float(len(df_out))) # and print the percentage of observations within each category
            print()

# Get categorical columns greater than 15 cat
multi_cat_cols = []
for col in df_out.columns:
    if df_out[col].dtypes =='O': # if variable  is categorical
        if len(df_out[col].unique())>15: # and has more than 10 categories
            multi_cat_cols.append(col)  # add to the list
            print(df_out.groupby(col)[col].count()/np.float(len(df_out))) # and print the percentage of observations within each category
            print()

# let's inspect our highly cardinal variable names
print(cat_cols_all)

# let's inspect our highly cardinal variable names
print(multi_cat_cols)
            
df_out.head()





# re-categorising under Others
# function to make the 2 rare value imputations at once if needed

def rare_imputation(df, variable):
    
    # find the most frequent category
    frequent_cat = df.groupby(variable)[variable].count().sort_values().tail(1).index.values[0]
    
    # find rare labels
    temp = df.groupby([variable])[variable].count()/np.float(len(df))
    rare_cat = [x for x in temp.loc[temp<0.05].index.values]
    
    # create new variables, with Rare labels imputed
    
    # by the most frequent category
    #df[variable+'_freq_imp'] = np.where(df[variable].isin(rare_cat), frequent_cat, df[variable])
    #X_test[variable+'_freq_imp'] = np.where(X_test[variable].isin(rare_cat), frequent_cat, X_test[variable])
    
    # by adding a new label 'Rare'
    df[variable] = np.where(df[variable].isin(rare_cat), 'Autres', df[variable])
    
# let's go ahead and impute rare categories
for col in cat_cols_all:
    rare_imputation(df_out, col)
    

# Get numerical columns
numerical_columns = df_out.select_dtypes(include=[np.number]).columns.tolist()

def get_OHE(df):
    df_OHE = pd.concat([df[numerical_columns], 
                         pd.get_dummies(df[cat_cols_all], drop_first=True)],
                        axis=1
                       )
    # drop the target
     #df_OHE.drop(['CA par place', 'dept_nom_Autres', 'Region_nom_Autres', 'Tarification_PSU', 'Réseau_CAT', 'Stock_Autres', 'Type crèche pd_MICRO'], axis=1, inplace=True)
    df_OHE.drop(['CA par place'], axis=1, inplace=True)
    return df_OHE

df_out_OHE = get_OHE(df_out)


# divide dataset into train and test set

X_train, X_test, y_train, y_test = train_test_split(df_out_OHE, df_out['CA par place'], test_size=0.20,
                                                    random_state=20)
print(X_train.shape, X_test.shape)

X_train.head(2)


def feature_selection_pipeline(X_train, X_test):
	### remove constant features
	constant_features = [feat for feat in X_train.columns if X_train[feat].std() == 0]

	X_train.drop(labels=constant_features, axis=1, inplace=True)
	X_test.drop(labels=constant_features, axis=1, inplace=True)

	# Print shape after removing constant features
	print(X_train.shape, X_test.shape)
    
### remove quasi-constant features
	sel = VarianceThreshold(
		threshold=0.01)  # 0.1 indicates 99% of observations approximately

	sel.fit(X_train)  # fit finds the features with low variance

	sum(sel.get_support()) # how many not quasi-constant?

	features_to_keep = X_train.columns[sel.get_support()]

	# we can then remove the features like this
	X_train = sel.transform(X_train)
	X_test = sel.transform(X_test)
	# Print shape after removing quasi-constant features
	print(X_train.shape, X_test.shape)

	# sklearn transformations lead to numpy arrays
	# transform the arrays back to dataframes & getting the columns assigned correctly

	X_train= pd.DataFrame(X_train)
	X_train.columns = features_to_keep

	X_test= pd.DataFrame(X_test)
	X_test.columns = features_to_keep
    
	### check for duplicated features in the training set
	duplicated_feat = []
	for i in range(0, len(X_train.columns)):
		if i % 10 == 0:  # this helps understand how the loop is going
			#print(i)
			col_1 = X_train.columns[i]

		for col_2 in X_train.columns[i + 1:]:
			if X_train[col_1].equals(X_train[col_2]):
				duplicated_feat.append(col_2)
				
	len(duplicated_feat)

	# remove duplicated features
	X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
	X_test.drop(labels=duplicated_feat, axis=1, inplace=True)

	# Print shape after removing duplicated features
	print(X_train.shape, X_test.shape)

	### find and remove correlated features
	def correlation(dataset, threshold):
		col_corr = set()  # Set of all the names of correlated columns
		corr_matrix = dataset.corr()
		for i in range(len(corr_matrix.columns)):
			for j in range(i):
				if abs(corr_matrix.iloc[i, j]) > threshold: #interested in absolute coeff value
					colname = corr_matrix.columns[i]  # getting the name of column
					col_corr.add(colname)
		return col_corr

	corr_features = correlation(X_train, 0.75)
	print('correlated features: ', len(set(corr_features)) )

	X_train.drop(labels=corr_features, axis=1, inplace=True)
	X_test.drop(labels=corr_features, axis=1, inplace=True)

	# Print shape after removing remove correlated features
	print(X_train.shape, X_test.shape)
	return X_train, X_test

X_train_corr, X_test_corr = feature_selection_pipeline(X_train, X_test)



# step forward feature selection

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sffs = SFS(LinearRegression(), 
           k_features=20, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='r2',
           cv=3)

sffs = sffs.fit(np.array(X_train_corr), y_train)

sffs.k_feature_idx_
selected_feat_sffs= X_train_corr.columns[list(sffs.k_feature_idx_)]
selected_feat_sffs
print('Selected features using stepwise forward feature selection: ', selected_feat_sffs)



# step backward feature selection
# indicated to select 15 features from the total, and that I want to select those features based on the optimal r2

def backward_feature_selection(X_train, X_test, y_train):
	sbfs = SFS(LinearRegression(n_jobs=4), 
			   k_features=20, 
			   forward=False, 
			   floating=False, 
			   verbose=2,
			   scoring='r2',
			   cv=3)

	sbfs = sbfs.fit(np.array(X_train.fillna(0)), y_train)
	selected_feat= X_train.columns[list(sbfs.k_feature_idx_)]
	print('Selected features using stepwise backward feature selection: ', selected_feat)
	X_train = X_train[selected_feat]
	X_test= X_test[selected_feat]
	print(X_train.shape, X_test.shape)
	return X_train, X_test

X_train_sbfs, X_test_sbfs = backward_feature_selection(X_train_corr, X_test_corr, y_train)


def train_rf(X_train, y_train, X_test, y_test, columns):
    # function to train the random forest
    # and test it on train and test sets
    
    #rf = RandomForestRegressor(n_estimators=800, random_state=39)
    rf = LinearRegression()
    
    if type(columns)==str: # if we train using only 1 variable (pass a string instead of list in the "columns" argument of the function)
        rf.fit(X_train[columns].to_frame(), y_train.values)
        pred_train = rf.predict(X_train[columns].to_frame())
        pred_test = rf.predict(X_test[columns].to_frame())
        
    else: # if we train using multiple variables (pass a list in the argument "columns")
        rf.fit(X_train[columns], y_train.values)
        pred_train = rf.predict(X_train[columns])
        pred_test = rf.predict(X_test[columns])
        
    print('Train set')
    print('mse: {}'.format(mean_squared_error(y_train, pred_train)))
    print('Test set')
    print('mse: {}'.format(mean_squared_error(y_test, pred_test)))
    
    
#Normaliser
#scaled_features_full = MinMaxScaler().fit_transform(X_full)
#X_scaled_full = pd.DataFrame(scaled_features_full, index=X_full.index, columns=X_full.columns)

# for LINEAR regression features need to be in the same scale
scaler = MinMaxScaler().fit(X_train_sbfs)
#scaler = StandardScaler().fit(X_train_sbfs)
X_train_sbfs_scaled =  pd.DataFrame(scaler.transform(X_train_sbfs),index=X_train_sbfs.index, columns=X_train_sbfs.columns)
X_test_sbfs_scaled =  pd.DataFrame(scaler.transform(X_test_sbfs),index=X_test_sbfs.index, columns=X_test_sbfs.columns)

train_rf(X_train_sbfs_scaled, y_train, X_test_sbfs_scaled, y_test, X_train_sbfs.columns)




selected_columns = [ 
       'Pop',
         'Rés princ',
        'Rés princ',
        'Fam']

X_train_sbfs_selected = X_train[selected_columns]

X_test_sbfs_selected = X_test[selected_columns]

scaler = MinMaxScaler().fit(X_train_sbfs_selected)
#scaler = StandardScaler().fit(X_train_sbfs_selected)
X_train_sbfs_scaled =  pd.DataFrame(scaler.transform(X_train_sbfs_selected),index=X_train_sbfs_selected.index, columns=X_train_sbfs_selected.columns)
X_test_sbfs_scaled =  pd.DataFrame(scaler.transform(X_test_sbfs_selected),index=X_test_sbfs_selected.index, columns=X_test_sbfs_selected.columns)





from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
def lin_reg(X_train, X_test, Y_train, Y_test ):  
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)
    linreg = LinearRegression()
    linreg.fit(X_train, Y_train)
    y_hat = linreg.predict(X_test)
    y_hat_train = linreg.predict(X_train)
    print('R_squared Score:', linreg.score(X_train, Y_train))
    #Display errors
    print('Mean Absolute Error:', mean_absolute_error(Y_test, y_hat))
    print('Root Mean Squared Error test:', mean_squared_error(Y_test, y_hat))
    print('Root Mean Squared Error train:', mean_squared_error(Y_train, y_hat_train))
    #Compare predicted and actual values
    print('Mean Predicted CA par place:', y_hat.mean())
    print('Mean CA par place:', Y_test.mean())

    errors = abs(y_hat - Y_test)
    mape = 100 * np.mean(errors / Y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return linreg

lin_reg(X_train_sbfs_scaled, X_test_sbfs_scaled, y_train, y_test )



from sklearn.feature_selection import f_regression
# f_regression finds the F-statistics for the *simple* regressions created with each of the independent variables
# In our case, this would mean running a  linear regression on  independent variable
# The limitation of this approach is that it does not take into account the mutual effect of the two features
f_regression(X_train_sbfs_scaled,y_train)
p_values = f_regression(X_train_sbfs_scaled,y_train)[1]
p_values.round(3)

linreg = LinearRegression()
linreg.fit(X_train_sbfs_scaled,y_train)
# If we want to find the Adjusted R-squared we can do so by knowing the r2, the # observations, the # features
r2 = linreg.score(X_train_sbfs_scaled,y_train)
# Number of observations is the shape along axis 0
n = X_train_sbfs_scaled.shape[0]
# Number of features (predictors, p) is the shape along axis 1
p = X_train_sbfs_scaled.shape[1]

# We find the Adjusted R-squared using the formula

adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
#adjusted_r2

y_pred_lr_train = linreg.predict(X_train_sbfs_scaled)
y_pred_lr_test = linreg.predict(X_test_sbfs_scaled)

# Let's create a new data frame with the names of the features
regr_summary = pd.DataFrame(data = X_train_sbfs_scaled.columns.values, columns=['Features'])

# Then we create and fill a second column, called 'Coefficients' with the coefficients of the regression
regr_summary ['Coefficients'] = linreg.coef_
regr_summary ['Intercept'] = linreg.intercept_
# Finally, we add the p-values we just calculated
regr_summary ['p-values'] = p_values.round(3)
regr_summary ['Adjusted R2'] = adjusted_r2.round(3)
regr_summary ['R2_train'] = metrics.r2_score(y_train, y_pred_lr_train).round(3)
regr_summary ['R2_test'] = metrics.r2_score(y_test, y_pred_lr_test).round(3)
#regr_summary ['R2_lr'] = linreg.score(X_train,Y_train)


# calculate these metrics by hand!
from sklearn import metrics

print('Train MAE:', metrics.mean_absolute_error(y_train, y_pred_lr_train))
print('Train MSE:', metrics.mean_squared_error(y_train, y_pred_lr_train))
print('Train RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_lr_train)))

print('Test MAE:', metrics.mean_absolute_error(y_test, y_pred_lr_test))
print('TestMSE:', metrics.mean_squared_error(y_test, y_pred_lr_test))
print('Test RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_lr_test)))
regr_summary


# predictions
y_pred_lr = linreg.predict(X_test_sbfs_scaled)

d = {'Crèche': X_test_sbfs_scaled.index,
    'true' : list(y_test),
     'predicted' : pd.Series(y_pred_lr),
     'ECART' : (list(y_test) - pd.Series(y_pred_lr))
    }

print(pd.DataFrame(d))


from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_lr))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_lr))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_lr)))  



c = '#E8B836'
# residual plot
plt.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds": linreg.predict(X_test_sbfs_scaled), "true": y_test})


preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals", kind = "scatter", color = c)

lm = LinearRegression()
model = lm.fit(X_train_sbfs_scaled, y_train)


from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
# Make cross validated predictions
predictions = cross_val_predict(model, df_out_OHE[selected_columns], df_out['CA par place'], cv=20)
plt.scatter(df_out['CA par place'], predictions)

# Perform 6-fold cross validation
scores = cross_val_score(model, df_out_OHE[selected_columns], df_out['CA par place'], cv=20)
print ("Cross-validated scores:", scores)

accuracy = metrics.r2_score(df_out['CA par place'], predictions)
print ("Cross-Predicted Accuracy:", accuracy)



def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy


LinearRegression_model = LinearRegression()
LinearRegression_model.fit(X_train_sbfs_scaled, y_train)
base_accuracy = evaluate(LinearRegression_model, X_test_sbfs_scaled, y_test)


Lasso_model = Lasso()
Lasso_model.fit(X_train_sbfs_scaled, y_train)
Lasso_accuracy = evaluate(Lasso_model, X_test_sbfs_scaled, y_test)

Ridge_model = Ridge()
Ridge_model.fit(X_train_sbfs_scaled, y_train)
Ridge_accuracy = evaluate(Ridge_model, X_test_sbfs_scaled, y_test)

ElasticNet_model = ElasticNet()
ElasticNet_model.fit(X_train_sbfs_scaled, y_train)
ElasticNet_accuracy = evaluate(ElasticNet_model, X_test_sbfs_scaled, y_test)

RandomForest_model = RandomForestRegressor()
RandomForest_model.fit(X_train_sbfs_scaled, y_train)
RandomForest_accuracy = evaluate(RandomForest_model, X_test_sbfs_scaled, y_test)


print('Improvement Lasso of {:0.2f}%.'.format( 100 * (Lasso_accuracy - base_accuracy) / base_accuracy))
print('Improvement Ridge of {:0.2f}%.'.format( 100 * (Ridge_accuracy - base_accuracy) / base_accuracy))
print('Improvement ElasticNet of {:0.2f}%.'.format( 100 * (ElasticNet_accuracy - base_accuracy) / base_accuracy))
print('Improvement RandomForest of {:0.2f}%.'.format( 100 * (RandomForest_accuracy - base_accuracy) / base_accuracy))


def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_model = DecisionTreeRegressor(random_state = 42)
base_model.fit(X_train_corr, y_train)
base_accuracy = evaluate(base_model, X_test_corr, y_test)


best_random = RandomForestRegressor(n_estimators = 10)
best_random.fit(X_train_corr, y_train)
random_accuracy = evaluate(best_random, X_test_corr, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))




# Create a random forest classifier
clf = RandomForestRegressor(n_estimators=10)

# Train the classifier
clf.fit(X_train_corr, y_train)

feat_labels = X_train_corr.columns

# Print the name and gini importance of each feature
for feature in zip(feat_labels, clf.feature_importances_):
    print(feature)

    
    
## GradientBoosting feature imporatance
regr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, max_depth=5)
regr.fit(X_train_corr, y_train)

feature_importance = regr.feature_importances_*100
rel_imp = pd.Series(feature_importance, index=X_train_corr.columns).sort_values(inplace=False)
rel_imp.T.plot(kind='barh', color='r', figsize=(15,15))
plt.xlabel('Variable Importance', size=10)
plt.gca().legend_ = None

# first ten importances 
importances = regr.feature_importances_[:10]
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices],  align = 'center')
plt.yticks(range(len(indices)), X_train_corr)
plt.xlabel('Relative Importance')
plt.show()




# Test slope and intercept
from statistics import mean


df_temp =  df_fin[(df_fin[ 'Rés princ logé gratuit'] > 0)]
#df_temp =  df[(df['Actif occ 15 ans ou plus travaille commune résidence'] < 100)]

#df[(df != 0).all(1)]
df_X_org = df_temp[ 'Rés princ logé gratuit']
df_y_org = df_temp['CA par place']

df_X_log = df_temp['Rés princ logé gratuit'].apply(np.log)
df_y_log = df_temp['CA par place'].apply(np.log)
#df_temp['Actif occ 15 ans ou plus travaille commune résidence'].apply(np.log).hist()
#df_temp['CA par place'].hist()
df_temp['CA par place'].hist()
def best_fit_slope_and_intercept(xs,ys):
    m = (((mean(xs)*mean(ys)) - mean(xs*ys)) /
         ((mean(xs)*mean(xs)) - mean(xs*xs)))
    
    b = mean(ys) - m*mean(xs)
    
    return m, b

m, b = best_fit_slope_and_intercept(df_X_log, df_y_log)

print(m,b)

regression_line = [(m*x)+b for x in df_X_log]
regression_line = []
for x in df_X_log:
    regression_line.append((m*x)+b)
    

from matplotlib import style
style.use('ggplot')

plt.scatter(df_X_log, df_y_log,color='#003F72')
plt.plot(df_X_log, regression_line)
plt.show()

